In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pandas as pd
import seaborn as sns
import multiprocessing as mp
from src.data_processing.processors.guppy_processors.experimental_metadata import ExperimentMetaData, MetaDataFactory
from src.data_processing.processors.guppy_processors.config_loader import ConfigLoader
from src.data_processing.processors.guppy_processors.processing_strategies import ProcessingStrategy, BehaviorProcessingStrategy, PhotometryProcessingStrategy
from src.data_processing.processors.guppy_processors.aggregators import BehaviorAggregationStrategy, PhotometryAggregationStrategy, aggregate_data
from src.data_processing.processors.guppy_processors.batch_data_processors import BatchBehaviorDataPreprocessor, BatchPhotometryDataPreprocessor
total_cores = mp.cpu_count()
print(f"Total number of cores: {total_cores}")


EXPERIMENTAL_CONFIGS = '/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Mike/LHA_dopamine/LH_NAC_Headfix_FP/Photometry/Pav_Training/claire_cohort/conf/config.yaml'
configs = ConfigLoader(EXPERIMENTAL_CONFIGS)
meta_data_factory = MetaDataFactory(configs)

Total number of cores: 12


In [ ]:
#behavior batch processing
behavior_strategy_params={
    'config_key': 'behavioral_events',
    'time_window': (-10, 20),
    'events': (('cue', 'lick'), 
               ('cue', 'encoder'),
               ('reward', 'lick'),
               ('reward', 'encoder')
               )

}


behavior_strategy = BehaviorProcessingStrategy(**behavior_strategy_params)
batch_behavior_preprocessor = BatchBehaviorDataPreprocessor(
    meta_data_factory, behavior_strategy)

batch_behavior_preprocessor.process_data(num_processors=10)
aggregate_data(configs, BehaviorAggregationStrategy)

In [10]:


# photometry_processing_params = {
#     'signal_correction': 'z_score',
#     'events_to_exclude': ['encoder', 'lick'],
#     'save_bool': True,
#     'return_df_bool': False
# }
# photometry_processing_strategy = PhotometryProcessingStrategy(
#     **photometry_processing_params)

# batch_photometry_preprocessor = BatchPhotometryDataPreprocessor(
#     meta_data_factory, photometry_processing_strategy)

# batch_photometry_preprocessor.process_data(num_processors=10)
aggregate_data(configs =configs, aggregation_strategy=PhotometryAggregationStrategy)

sys:1: UserWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


In [ ]:
df = pd.read_parquet('/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Mike/LHA_dopamine/LH_NAC_Headfix_FP/Photometry/Pav_Training/claire_cohort/analyzed_data/aggregated_data/aggregated_behavior_data.parquet').sort_values(by = 'date')

In [ ]:
facet = sns.FacetGrid(data=df, col = 'date', hue='subject', height=5, col_wrap=5, aspect=1)
facet.map(sns.lineplot,  "time", "lick_aligned_to_cue", errorbar='se')

In [ ]:
facet = sns.FacetGrid(data=df, col='date')
facet.map(sns.lineplot,  "time", "encoder_aligned_to_cue", errorbar='se')

In [ ]:
from matplotlib import pyplot as plt

facet = sns.FacetGrid(data=data.filter(pl.col('behavioral_events') == 'cue'),
                      col='structures', 
                      height=5, 
                      aspect=1, 
                      row_order=['LHA', 'NAC'], 
                      hue='structures')
facet = facet.map(sns.lineplot,  "timestamps", "z_score", errorbar='se')
facet.map(plt.axvline, x=0, color='grey', linewidth=0.5)